In [33]:
import ee
ee.Initialize()

In [34]:
import ee
import geemap
import os
import functools

In [35]:
# Function to get a square around point of interest
# Rural : 5.5km Radius
# Urban : 2 km Radius
def bounding_box(loc, urban_rural):
    if urban_rural is 'U':
        size = 1000
        print('Urban Areas because', cluster['urban_rural'])
    else:
        size = 5000
        print('Rural Areas because', cluster['urban_rural'])

    intermediate_buffer = loc.buffer(size) #buffer radius, half your box width in m
    intermediate_box = intermediate_buffer.bounds() #Draw a bounding box around the circle
    return(intermediate_box)

In [36]:
def maskClouds(img, MAX_CLOUD_PROBABILITY):
    #MAX_CLOUD_PROBABILITY = 65
    clouds = ee.Image(img.get('cloud_mask')).select('probability')
    isNotCloud = clouds.lt(MAX_CLOUD_PROBABILITY)
    return img.updateMask(isNotCloud)

In [37]:
def maskEdges(s2_img):
  return s2_img.updateMask(s2_img.select('B8A').mask().updateMask(s2_img.select('B9').mask()))

In [38]:
def get_image(cluster):
    
    print('Getting cluster', cluster['ID'])
    s2Sr = ee.ImageCollection('COPERNICUS/S2_SR');
    s2Clouds = ee.ImageCollection('COPERNICUS/S2_CLOUD_PROBABILITY')

    #Define parameters
    START_DATE = ee.Date('2019-01-01')
    END_DATE = ee.Date('2019-03-31')
    MAX_CLOUD_PROBABILITY = 20
   
    #Point of interest (longitude, latidude)
    loc = ee.Geometry.Point([cluster["longitude"], cluster["latidude"]])
    #Region of interest
    region = bounding_box(loc, cluster['urban_rural'])

    # Filter input collections by desired data range and region.
    #criteria = ee.Filter.And(ee.Filter.bounds(region), ee.Filter.date(START_DATE, END_DATE))
    #s2Sr = s2Sr.filter(criteria).map(maskEdges)
    #s2Clouds = s2Clouds.filter(criteria)
    s2Sr = s2Sr.filterBounds(region).filterDate(START_DATE, END_DATE).map(maskEdges)
    s2Clouds = s2Clouds.filterBounds(region).filterDate(START_DATE, END_DATE)

    # Join S2 SR with cloud probability dataset to add cloud mask.
    s2SrWithCloudMask = ee.Join.saveFirst('cloud_mask').apply(
      primary =  s2Sr, 
      secondary = s2Clouds, 
      condition = ee.Filter.equals(
          leftField =  'system:index', rightField = 'system:index') 
        )

    maskCloudsWithProb = functools.partial(maskClouds, MAX_CLOUD_PROBABILITY=MAX_CLOUD_PROBABILITY)
    s2CloudMasked = ee.ImageCollection(s2SrWithCloudMask).map(maskCloudsWithProb).median()
    s2CloudMasked = s2CloudMasked.select(['B4','B3','B2']).clip(region)

    out_dir = os.path.join(os.path.expanduser('~'), '/home/shannon/Dokumente/Dokumente/studium/ASA/Projekt/SatteliteImage__GEE/MZ007')
    filename = os.path.join(out_dir, cluster['ID']+'.tif')
    print(filename)
    geemap.ee_export_image(s2CloudMasked, filename=filename, scale=10)
    
    return loc

#Point of interest
#loc = ee.Geometry.Point([11.557766, 48.140458])
#Region of interest
#roi = bounding_box(loc)

collection = ee.ImageCollection('COPERNICUS/S2_SR') \
    .filterBounds(roi) \
    .filterDate('2020-01-01', '2020-01-31')

#Direction to save it to
out_dir = os.path.join(os.path.expanduser('~'), 'Downloads')
#Select RGB channel to be displayed in the images and clip the whole image to region of interest.
coll_rgb = collection.select(['B4','B3','B2']).map(lambda i: i.clip(roi))

print(collection.aggregate_array('system:index').getInfo())

#Export images to defined region
geemap.ee_export_image_collection(coll_rgb, out_dir=out_dir, scale=10)


In [39]:
from dbfread import DBF

file = "/home/shannon/Dokumente/Dokumente/studium/ASA/Projekt/SatteliteImage__GEE/MZGE61FL-2.dbf"
clusters = list()

for record in DBF(file):
    #print(record, "\n")
    #print(record['URBAN_RURA'], "\n")
    cluster = {"ID" : record['DHSID'], "cluster": record['DHSCLUST'],\
               "urban_rural": record['URBAN_RURA'], "latidude": record["LATNUM"],\
              "longitude": record['LONGNUM']}
    clusters.append(cluster)
print(cluster)

{'ID': 'MZ201100000611', 'cluster': 611.0, 'urban_rural': 'U', 'latidude': -26.011408, 'longitude': 32.931726}


In [ ]:
map1 = geemap.Map()

for cluster in clusters:
    loc = get_image(cluster)
    map1.addLayer(loc)


In [ ]:
map1.centerObject(loc, 8)
map1    